In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

%matplotlib inline
import matplotlib
%config InlineBackend.figure_format = 'jpg'
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")
import matplotlib.pyplot as plt

Branje podatkov

In [ ]:
netflix = pd.read_csv("Data\\netflix_titles.csv")

movies = pd.read_csv("Data\\akas.csv")
ratings = pd.read_csv("Data\\ratings.csv")
names = pd.read_csv("Data\\names_basic.csv")

crew=pd.read_csv("Data\\crew.csv")
crew.rename(columns = {'tconst':'titleId'}, inplace = True) 

director_names_and_stats=pd.read_csv("Data\\namebasics.csv")
director_names_and_stats.rename(columns = {'tconst':'titleId'}, inplace = True) 

In [ ]:
ratings = ratings.rename(columns = {'tconst':'titleId'})
originals = movies[movies["types"] == "original"]
originals = originals[["titleId", "title"]]
originals = originals.merge(ratings, on="titleId")

projekt = originals.merge(crew[["titleId","directors"]], on="titleId")

originals = originals.rename(columns = {'tconst':'title'})
netflix = netflix.merge(originals, on="title")


In [ ]:
def najdi(table):
    if table.empty:
        return table
    else:
        return table.iloc[table["numVotes"].argmax()]

def getMovieDirID(MovieName):
    for index, row in projekt.iterrows():
        if (projekt.loc[index, row["titleId"]] == MovieName):
            return projekt.loc[index, row["directors"]]

def getDirector(dirID):
    df=director_names_and_stats[["nconst","primaryName"]]
    for index, row in df.iterrows():     
        if(df.loc[index, row["nconst"]] == dirID):
            return df.loc[index,"primaryName"]
            

In [ ]:
netflix1=netflix.fillna(0)

In [ ]:
i = 0
for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    table = originals[originals["title"] == naslov]
    if table.empty:
        pass
    else:
        a = najdi(originals[originals["title"] == naslov])
        netflix1.loc[idx, "averageRating"] = a["averageRating"]
        netflix1.loc[idx, "numVotes"] = a["numVotes"]

Preberemo imdb bazo za mankajoce podatke

In [ ]:
from imdb import IMDb
from imdb import IMDbDataAccessError

movs = IMDb()
i = 0

for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    if netflix1.loc[idx, "director"] == "0":
        m = movs.search_movie(naslov)
        if len(m) > 0:
            try:
                data = movs.get_movie(m[0].movieID)
                netflix1.loc[idx, "director"] = data["director"][0]['name']
            except KeyError as e:
                netflix1.loc[idx, "director"] = "-1"
            except (URLError, IMDbDataAccessErro) as e:
                print(data)
    if i%100 == 0:
        print(i)

In [77]:
netflix1 = pd.read_csv("Netflix_all.csv")
#netflix1['date_added'].to_csv("D:\\OneDrive\\PhenixZeroPrime\FRI\\2. Letnik 2. Semester\\Podatkovno Rudarjenje\\PR_Projektna\\PR19KZMBG\\datumi.csv", index=False)
netflixWithoutMissingDate = netflix1[netflix1['date_added'] != '0']

Popravilo datumov

In [78]:
dic = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12', '':'0'}
netflixWithoutMissingDate['month_added'] = 0
netflixWithoutMissingDate['year_added'] = 0
#September 9, 2019
for index, row in netflix1.iterrows():
    if(row['date_added'] != "0"):  
        monthDay_year = row['date_added'].split(", ") #[September 9, 2019]
        month_day = monthDay_year[0].split(" ") #[September, 9]

        if(len(month_day) == 2): #Če je split enak: month_day == [September, 9]
            netflixWithoutMissingDate.loc[index, "date_added"] = "%s/%s/%s" % (dic[month_day[0]], month_day[1], monthDay_year[1])

            try:
                netflixWithoutMissingDate.loc[index, 'date_added'] = pd.to_datetime(netflixWithoutMissingDate.loc[index, 'date_added'], format="%m/%d/%Y")
            except (TypeError, ValueError) as e:
                print(netflixWithoutMissingDate.loc[index, 'date_added'])
            
            netflixWithoutMissingDate.loc[index, "month_added"]= month_day[0]
            netflixWithoutMissingDate.loc[index, "year_added"]= monthDay_year[1]

        elif(len(month_day) == 3): #Če je split napačen: month_day == ['', September, 9]
            netflixWithoutMissingDate.loc[index, "date_added"] = "%s/%s/%s" % (dic[month_day[1]], month_day[2], monthDay_year[1])
            
            try:
                netflixWithoutMissingDate.loc[index, 'date_added'] = pd.to_datetime(netflixWithoutMissingDate.loc[index, 'date_added'], format="%m/%d/%Y")
            except (TypeError, ValueError) as e:
                print(netflixWithoutMissingDate.loc[index, 'date_added'])
                print(month_day[1])
            
            netflixWithoutMissingDate.loc[index, "month_added"]= month_day[1]
            netflixWithoutMissingDate.loc[index, "year_added"]= monthDay_year[1]

In [79]:
netflixWithoutMissingDate = netflixWithoutMissingDate.astype({'year_added': 'int64'})
netflixWithoutMissingDate['date_added']= pd.to_datetime(netflixWithoutMissingDate['date_added']) 

In [80]:
netflixWithoutMissingDate.dtypes

show_id                   int64
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
averageRating           float64
numVotes                float64
month_added              object
year_added                int64
dtype: object

In [82]:

netflixWithoutMissingDate.tail(29)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,averageRating,numVotes,month_added,year_added
6194,80046347,TV Show,Señora Acero,-1,"Blanca Soto, Litzy, Damián Alcázar, Marco Pére...","Mexico, United States",2019-04-28,2018,TV-14,5 Seasons,"Crime TV Shows, International TV Shows, Spanis...","When her drug-trafficking husband dies, an una...",7.2,232.0,April,2019
6195,80136790,TV Show,Goodnight DJ 1,Jake Paltrow,"Chan Fong, Z-Chen, Frederick Lee, Melvin Sia, ...",0,2018-04-27,2016,TV-14,2 Seasons,"International TV Shows, TV Dramas, TV Horror",Young radio announcers from varying background...,5.8,8113.0,April,2018
6196,80127464,TV Show,Mar de Plástico,-1,"Rodolfo Sancho, Belén López, Pedro Casablanc, ...",Spain,2017-04-27,2017,TV-MA,2 Seasons,"Crime TV Shows, International TV Shows, Spanis...",In a town in southern Spain where racial tensi...,7.2,1202.0,April,2017
6197,80987209,TV Show,"Rica, Famosa, Latina",-1,0,United States,2019-04-26,2016,TV-14,4 Seasons,"International TV Shows, Reality TV, Spanish-La...",This reality series follows the lives of a gro...,5.7,46.0,April,2019
6198,80189829,TV Show,The Protector,-1,"Çağatay Ulusoy, Ayça Ayşin Turan, Hazar Ergüçl...",Turkey,2019-04-26,2019,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",Discovering his ties to a secret ancient order...,6.9,28150.0,April,2019
6199,80244228,TV Show,Beauty and the Bitches,Johnny Parker,"Methinee Kingpayome, Son Songpaisarn, Yingying...",0,2019-04-23,2016,TV-14,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",Beauty pageant contestants from all over Thail...,-1.0,-1.0,April,2019
6200,80018869,TV Show,Liv and Maddie,-1,"Dove Cameron, Joey Bragg, Tenzing Norgay Train...",United States,2017-04-23,2017,TV-G,4 Seasons,"Kids' TV, TV Comedies",After four years in Hollywood as a television ...,6.3,6073.0,April,2017
6201,80050007,TV Show,The Mr. Peabody and Sherman Show,-1,"Chris Parnell, Max Charles, Dieter Jansen, Da'...",United States,2017-04-21,2017,TV-Y7,4 Seasons,"Kids' TV, TV Comedies",Mr. Peabody and Sherman host a zany late-night...,5.6,285.0,April,2017
6202,80116626,TV Show,The Cravings,-1,"Park Hee-bon, Phillip Choi, Hong Wan-pyo, Choi...",South Korea,2017-04-20,2016,TV-14,2 Seasons,"International TV Shows, Korean TV Shows, TV Co...",Thirtysomething career woman Jae-yeong attempt...,7.2,50.0,April,2017
6203,70158329,TV Show,Star Trek: The Next Generation,-1,"Patrick Stewart, Jonathan Frakes, LeVar Burton...",United States,2018-04-02,1993,TV-PG,7 Seasons,"TV Action & Adventure, TV Sci-Fi & Fantasy",Decades after the adventures of the original E...,8.6,93178.0,April,2018


tu nevemo kaj nrdi

In [ ]:
#netflix1.to_csv("/Users/gregorsink/Desktop/PR Projekt/PR19KKZDBMPBPGS/Netflix_all.csv", index=False)
netflixContentByYears = netflix1.groupby(['year_added'])['year_added'].agg(['count'])
netflixContentByYears = netflixContentByYears.reset_index()
netflixContentByYears.drop([0])

#netflixContentByYears.set_index('index')

In [ ]:
#netflix1 = netflix1.sort_values(by='date_added', ascending=False)
#netflix1 = pd.to_datetime(netflix1['date_added'], dayfirst=True)
#netflix1['date_added'] = pd.to_datetime(netflix1['date_added']).dt.year
#netflix1['Year'] = pd.DatetimeIndex(netflix1['date_added']).month 
#tv_show = df[df["type"] == "TV Show"]
#movies = df[df["type"] == "Movie"]
for index, row in netflix1.iterrows():
    if(row['date_added'] != '0'):
        aaa = row['date_added']
        new = aaa.split("-")
        netflix1.loc[index, "month_added"]= new[1]
        netflix1[index, "year_added"]= new[2] 
        